In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

In [7]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [11]:
url = 'https://books.toscrape.com/'
browser.visit(url)

In [12]:
titles = []
for x in range(1, 51):

    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    titles_found = soup.find_all('h3')
    # products = soup.find_all('article', class_='product_pod')

    for product in titles_found:
        # print('page:', x, '-------------')
        titles.append(product.text)
        print(f"+Page: ", x)
    if browser.links.find_by_partial_text('next'):
        browser.links.find_by_partial_text('next').click()
    else:
        print("-Done")

A Light in the ...
Tipping the Velvet
Soumission
Sharp Objects
Sapiens: A Brief History ...
The Requiem Red
The Dirty Little Secrets ...
The Coming Woman: A ...
The Boys in the ...
The Black Maria
Starving Hearts (Triangular Trade ...
Shakespeare's Sonnets
Set Me Free
Scott Pilgrim's Precious Little ...
Rip it Up and ...
Our Band Could Be ...
Olio
Mesaerion: The Best Science ...
Libertarianism for Beginners
It's Only the Himalayas
In Her Wake
How Music Works
Foolproof Preserving: A Guide ...
Chase Me (Paris Nights ...
Black Dust
Birdsong: A Story in ...
America's Cradle of Quarterbacks: ...
Aladdin and His Wonderful ...
Worlds Elsewhere: Journeys Around ...
Wall and Piece
The Four Agreements: A ...
The Five Love Languages: ...
The Elephant Tree
The Bear and the ...
Sophie's World
Penny Maybe
Maude (1883-1993):She Grew Up ...
In a Dark, Dark ...
Behind Closed Doors
You can't bury them ...
Slow States of Collapse: ...
Reasons to Stay Alive
Private Paris (Private #10)
#HigherSelfie: Wake 

In [13]:
print(len(titles))

1000


In [14]:
browser.quit()